# Lab 03: TensorFlow vs. PyTorch
- Train a model on MNIST in both TensorFlow and PyTorch, convert to TFLite and ONNX.  
- Use tf.GradientTape for Tensorflow custom training loop.



## TensorFlow Implementation

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize pixel values to range [0, 1]
x_train = x_train / 255.0
x_test = x_test / 255.0

# One-hot encode the labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Build a simple feedforward neural network
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),             # Input shape matching MNIST image size
    tf.keras.layers.Flatten(),                         # Flatten 28x28 images to 1D vectors
    tf.keras.layers.Dense(64, activation='relu'),      # Hidden layer with 64 neurons and ReLU activation
    tf.keras.layers.Dense(10, activation='softmax')    # Output layer with 10 neurons (one per digit class)
])

# Compile the model with Adam optimizer and categorical crossentropy loss
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model and measure training time
start = time.time()
model.fit(x_train, y_train, epochs=5)
end = time.time()
print(f"TF Training time: {end-start:.2f} seconds")     # Print the training duration

# Evaluate the model on the test set
model.evaluate(x_test, y_test)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.8593 - loss: 0.5044
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9530 - loss: 0.1613
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9681 - loss: 0.1083
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9765 - loss: 0.0808
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9805 - loss: 0.0654
TF Training time: 29.23 seconds
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9698 - loss: 0.1041


[0.08797445148229599, 0.9739000201225281]

## Convert TensorFlow model to TFLite

In [2]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)

Saved artifact at '/tmp/tmpmyj6dfr7'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  132709103118160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132709100014672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132709103122000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132709100015056: TensorSpec(shape=(), dtype=tf.resource, name=None)


## PyTorch Implementation

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time

# Transform: convert to tensor and flatten 28x28 image to a 784-dimensional vector
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))  # Flatten the image
])

# Data loaders with batch sizes
train_loader = DataLoader(
    datasets.MNIST(root='./data', train=True, transform=transform, download=True),
    batch_size=32
)
test_loader = DataLoader(
    datasets.MNIST(root='./data', train=False, transform=transform, download=True),
    batch_size=1000
)

# Define a simple fully connected neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 128)   # Input layer: 784 input features (28x28), 128 hidden units
        self.fc2 = nn.Linear(128, 10)    # Output layer: 128 hidden units -> 10 output classes (digits 0-9)

    def forward(self, x):
        x = F.relu(self.fc1(x))          # Apply ReLU activation after first linear layer
        return self.fc2(x)               # Output layer without activation (CrossEntropyLoss handles softmax)

model = Net()
optimizer = optim.Adam(model.parameters())      # Adam optimizer for parameter updates
loss_fn = nn.CrossEntropyLoss()                 # Cross-entropy loss for classification

# Training loop with time measurement
start = time.time()
for epoch in range(5):                          # Train for 5 epochs
    for x, y in train_loader:
        optimizer.zero_grad()                   # Reset gradients
        pred = model(x)                         # Forward pass
        loss = loss_fn(pred, y)                 # Compute loss
        loss.backward()                         # Backpropagation
        optimizer.step()                        # Update weights
end = time.time()
print(f"PyTorch Training time: {end - start:.2f} seconds")  # Output training duration

# Evaluation loop
model.eval()
correct = 0
with torch.no_grad():                           # Disable gradient calculation during inference
    for x, y in test_loader:
        output = model(x)
        pred = output.argmax(1)                 # Get predicted class
        correct += (pred == y).sum().item()     # Count correct predictions
print(f"Test accuracy: {correct / len(test_loader.dataset):.4f}")  # Output test accuracy


100%|██████████| 9.91M/9.91M [00:01<00:00, 5.10MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 133kB/s]
100%|██████████| 1.65M/1.65M [00:06<00:00, 246kB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.41MB/s]


PyTorch Training time: 53.00 seconds
Test accuracy: 0.9724


## Convert PyTorch model to ONNX

In [4]:
# Install ONNX
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 119.0 MB/s eta 0:00:00


In [5]:
dummy_input = torch.randn(1, 784)
torch.onnx.export(model, dummy_input, "model.onnx",
                  input_names=["input"], output_names=["output"])

## TensorFlow custom training loop using tf.GradientTape

In [6]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train / 255.0    # Normalize pixel values to [0, 1]
x_test = x_test / 255.0      # Normalize pixel values to [0, 1]

# One-hot encode the labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Prepare datasets using tf.data API
batch_size = 32    # Use same batch size as in first TensorFlow example

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)) \
    .shuffle(10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Define a simple Sequential model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),           # Input shape matches MNIST image dimensions
    tf.keras.layers.Flatten(),                       # Flatten 28x28 into a 784-dim vector
    tf.keras.layers.Dense(64, activation='relu'),    # Hidden layer with 64 neurons and ReLU activation
    tf.keras.layers.Dense(10, activation='softmax')  # Output layer with 10 neurons (digit classes) and softmax
])

# Set up training components
loss_fn = tf.keras.losses.CategoricalCrossentropy()      # Categorical crossentropy for multi-class classification
optimizer = tf.keras.optimizers.Adam()                   # Adam optimizer
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()  # Training accuracy metric
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()   # Testing accuracy metric

# Custom training loop
epochs = 5
start = time.time()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch, training=True)       # Forward pass
            loss = loss_fn(y_batch, logits)              # Compute loss
        grads = tape.gradient(loss, model.trainable_variables)  # Compute gradients
        optimizer.apply_gradients(zip(grads, model.trainable_variables))  # Update weights
        train_acc_metric.update_state(y_batch, logits)   # Update training accuracy

        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy():.4f}, Accuracy: {train_acc_metric.result().numpy():.4f}")

    print(f"Training Accuracy for epoch {epoch + 1}: {train_acc_metric.result().numpy():.4f}")
    train_acc_metric.reset_state()
end = time.time()
print(f"\nTF Training time: {end - start:.2f} seconds")

# Custom evaluation loop
for x_batch, y_batch in test_dataset:
    test_logits = model(x_batch, training=False)     # Forward pass in inference mode
    test_acc_metric.update_state(y_batch, test_logits)  # Update test accuracy metric

print(f"Test Accuracy: {test_acc_metric.result().numpy():.4f}")



Epoch 1/5
Step 0, Loss: 2.3851, Accuracy: 0.0938
Step 100, Loss: 0.4746, Accuracy: 0.7262
Step 200, Loss: 0.4701, Accuracy: 0.8010
Step 300, Loss: 0.2824, Accuracy: 0.8278
Step 400, Loss: 0.2232, Accuracy: 0.8472
Step 500, Loss: 0.2614, Accuracy: 0.8610
Step 600, Loss: 0.3206, Accuracy: 0.8695
Step 700, Loss: 0.2948, Accuracy: 0.8763
Step 800, Loss: 0.2484, Accuracy: 0.8824
Step 900, Loss: 0.4715, Accuracy: 0.8872
Step 1000, Loss: 0.3232, Accuracy: 0.8909
Step 1100, Loss: 0.1217, Accuracy: 0.8939
Step 1200, Loss: 0.1202, Accuracy: 0.8970
Step 1300, Loss: 0.1591, Accuracy: 0.8996
Step 1400, Loss: 0.1314, Accuracy: 0.9019
Step 1500, Loss: 0.3380, Accuracy: 0.9045
Step 1600, Loss: 0.1207, Accuracy: 0.9070
Step 1700, Loss: 0.2029, Accuracy: 0.9094
Step 1800, Loss: 0.1129, Accuracy: 0.9114
Training Accuracy for epoch 1: 0.9128

Epoch 2/5
Step 0, Loss: 0.3366, Accuracy: 0.8750
Step 100, Loss: 0.1211, Accuracy: 0.9508
Step 200, Loss: 0.0795, Accuracy: 0.9515
Step 300, Loss: 0.4939, Accuracy:

## Performance Otimization with Graph Execution using @tf.function

In [7]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0   # Normalize pixel values to [0, 1]
x_test = x_test / 255.0     # Normalize pixel values to [0, 1]
y_train = to_categorical(y_train, num_classes=10)  # One-hot encode labels
y_test = to_categorical(y_test, num_classes=10)

# Prepare datasets
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Define model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),          # MNIST image shape
    tf.keras.layers.Flatten(),                      # Flatten to 784-dim vector
    tf.keras.layers.Dense(64, activation='relu'),   # Hidden layer: 64 neurons, ReLU activation
    tf.keras.layers.Dense(10, activation='softmax') # Output layer: 10 classes, softmax activation
])

# Define loss, optimizer, and metrics
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

@tf.function  # compile the function into a graph
def train_step(x_batch, y_batch):
    with tf.GradientTape() as tape:
        logits = model(x_batch, training=True)
        loss = loss_fn(y_batch, logits)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_acc_metric.update_state(y_batch, logits)
    return loss

# Training loop
epochs = 5
start = time.time()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        loss = train_step(x_batch, y_batch)

        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy():.4f}, Accuracy: {train_acc_metric.result().numpy():.4f}")

    print(f"Training Accuracy for epoch {epoch + 1}: {train_acc_metric.result().numpy():.4f}")
    train_acc_metric.reset_state()
end = time.time()
print(f"\nTF Training time: {end - start:.2f} seconds")

# Evaluation loop
for x_batch, y_batch in test_dataset:
    test_logits = model(x_batch, training=False)
    test_acc_metric.update_state(y_batch, test_logits)

print(f"Test Accuracy: {test_acc_metric.result().numpy():.4f}")



Epoch 1/5
Step 0, Loss: 2.3430, Accuracy: 0.0938
Step 100, Loss: 0.3737, Accuracy: 0.7169
Step 200, Loss: 0.3423, Accuracy: 0.7956
Step 300, Loss: 0.4230, Accuracy: 0.8300
Step 400, Loss: 1.0134, Accuracy: 0.8490
Step 500, Loss: 0.4091, Accuracy: 0.8620
Step 600, Loss: 0.3783, Accuracy: 0.8706
Step 700, Loss: 0.0889, Accuracy: 0.8768
Step 800, Loss: 0.2109, Accuracy: 0.8821
Step 900, Loss: 0.5630, Accuracy: 0.8871
Step 1000, Loss: 0.5430, Accuracy: 0.8916
Step 1100, Loss: 0.3470, Accuracy: 0.8948
Step 1200, Loss: 0.3125, Accuracy: 0.8979
Step 1300, Loss: 0.3782, Accuracy: 0.9012
Step 1400, Loss: 0.3207, Accuracy: 0.9044
Step 1500, Loss: 0.1462, Accuracy: 0.9065
Step 1600, Loss: 0.2320, Accuracy: 0.9095
Step 1700, Loss: 0.0461, Accuracy: 0.9116
Step 1800, Loss: 0.2461, Accuracy: 0.9139
Training Accuracy for epoch 1: 0.9154

Epoch 2/5
Step 0, Loss: 0.1203, Accuracy: 0.9688
Step 100, Loss: 0.3289, Accuracy: 0.9554
Step 200, Loss: 0.1423, Accuracy: 0.9524
Step 300, Loss: 0.4255, Accuracy: